In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np
import pickle

In [4]:
with open('../data/molecules.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [5]:
structures = pd.read_feather('../data/structures.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

In [6]:
molecules['dsgdb9nsd_000001'].bonds

[(0, 1, Bond(dist=1.0919529, valency=1, strength=411)),
 (0, 2, Bond(dist=1.0919516, valency=1, strength=411)),
 (0, 3, Bond(dist=1.0919464, valency=1, strength=411)),
 (0, 4, Bond(dist=1.0919476, valency=1, strength=411))]

In [ ]:
labelled.head()

# Inputs

* bond type
  * One hot encoding
* bond length
* atom1 position
* atom2 position
* atom3 position
* atom4 position

In [ ]:
def merge(data):
    data.copy()
    
    m0 = data.merge(structures, left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name', 'atom_index'], suffixes=('0', '0'))
    m1 = data.merge(structures, left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name', 'atom_index'], suffixes=('1', '1'))
    
    l0 = m0[['x', 'y', 'z']]
    l1 = m1[['x', 'y', 'z']]
    d = l0 - l1
    d2 = d * d
    dist2 = d2.x + d2.y + d2.z
    dist = dist2.apply(np.sqrt)
    dist.name = 'distance'
    
    merged = data.join(dist)
    merged['atom_0'] = m0.atom
    merged['atom_1'] = m1.atom
    
    return merged

In [ ]:
labelled_merged = merge(labelled)
unlabelled_merged = merge(unlabelled)

In [ ]:
min_dist = min(labelled_merged.distance.min(), unlabelled_merged.distance.min())
max_dist = max(labelled_merged.distance.max(), unlabelled_merged.distance.max())

min_coeff = labelled_merged.scalar_coupling_constant.min()
max_coeff = labelled_merged.scalar_coupling_constant.max()

In [ ]:
offset = min_coeff
scale = max_coeff - min_coeff

labelled_merged['norm_distance'] = (labelled_merged.distance - min_dist) / (max_dist - min_dist)
unlabelled_merged['norm_distance'] = (unlabelled_merged.distance - min_dist) / (max_dist - min_dist)

labelled_merged['norm_scc'] = (labelled_merged.scalar_coupling_constant - min_coeff) / (max_coeff - min_coeff)

In [ ]:
types = labelled.type.unique()
atoms = structures.atom.unique()

index = 0
type_index = {}
for t in types:
    type_index[t] = index
    index += 1

atom_index = {}
for a in atoms:
    atom_index[a] = index
    index += 2

dist_index = index
index += 1

columns = index

In [ ]:
def partition_labelled(data, count=None, train_frac=0.7):
    n_labelled = count if count is not None else len(labelled)
    n_train = int(n_labelled * train_frac)
    n_test = n_labelled - n_train
    indices = np.arange(0, n_labelled)
    np.random.shuffle(indices)
    
    train_indices = indices[0:n_train]
    test_indices = indices[n_train:]
    
    train = data.iloc[train_indices, :]
    test = data.iloc[test_indices, :]

    return train, test

def make_input(data, columns, type_index, atom_index, dist_index):
    n = len(data)
    input = np.zeros((columns, n), dtype='float32')
    
    for t in type_index:
        input[type_index[t], data.type == t] = 1
        
    for a in atom_index:
        input[atom_index[a], data.atom_0 == t] = 1
        input[atom_index[a] + 1, data.atom_1 == t] = 1
        
    input[dist_index] = data.norm_distance
        
    return input.T

def make_output(data):
    n = len(data)
    output = np.zeros(n, dtype='float32')
    output[:] = data.norm_scc
        
    return output

In [ ]:
from sklearn.svm import SVR,LinearSVR

In [ ]:
labelled_merged.columns

In [ ]:
labelled_train, labelled_test = partition_labelled(labelled_merged, 30000)

labelled_train_input = make_input(labelled_train, columns, type_index, atom_index, dist_index)
labelled_train_output = make_output(labelled_train)

labelled_test_input = make_input(labelled_test, columns, type_index, atom_index, dist_index)
labelled_test_output = make_output(labelled_test)

labelled_train_input.shape, labelled_train_output.shape

In [ ]:
model = LinearSVR()
model.fit(labelled_train_input, labelled_train_output)
print('')
#print(model.predict(test_input))
#print(test_output)
model.score(labelled_test_input, labelled_test_output)

In [ ]:
labelled_train_input = make_input(labelled_merged, columns, type_index, atom_index, dist_index)
labelled_train_output = make_output(labelled_merged)

labelled_train_input.shape, labelled_train_output.shape

In [ ]:
model = LinearSVR()
model.fit(labelled_train_input, labelled_train_output)

In [ ]:
unlabelled_input = make_input(unlabelled_merged, columns, type_index, atom_index, dist_index)

In [ ]:
unlabelled_output = model.predict(unlabelled_input)

In [ ]:
unlabelled_output

In [ ]:
unlabelled_output = unlabelled_output * scale + offset

In [ ]:
unlabelled_output

In [ ]:
output_df = pd.DataFrame({'id':unlabelled.id, 'scalar_coupling_constant':pd.Series(unlabelled_output, index=unlabelled.index)})

In [ ]:
output_df

In [ ]:
output_df.to_csv('../data/pred.csv',index=False)